In [2]:
import os

In [3]:
!pwd

/home/gourav/ML/Text_Classification_Model_Builder/research


In [4]:
os.chdir("../")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class DataProcessingConfig:
    model_name : str
    raw_data_dir: Path
    processed_data_dir: Path
    split_data_dir: Path

    text_col: str
    label_col: str

    train_data_size: float
    test_data_size: float
    val_data_size: float

In [10]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


In [11]:
class ConfigurationManager:

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_processing_config(self) -> DataProcessingConfig:

        config = self.config.data_processing
        params = self.params.data_processing


        data_processing_config = DataProcessingConfig(

            model_name = params.train_model.model_name,
            raw_data_dir= config.raw_data_dir,
            processed_data_dir= config.processed_data_dir,
            split_data_dir= config.split_data_dir,

            text_col= params.text_col,
            label_col= params.label_col,

            train_data_size= params.train_data_size,
            test_data_size= params.test_data_size,
            val_data_size= params.val_data_size,

        )

        return data_processing_config
       

In [13]:
import os
import pandas as pd
from src.utils.common import join_path
from transformers import AutoTokenizer

/home/gourav/ML/Text_Classification_Model_Builder/tc_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
class DataProcessing:
    def __init__(self,config = DataProcessingConfig) -> None:
        self.config = config


    def get_encoded_labels(self, data):
        
        result_data = data.astype('category').cat.codes
        return result_data


    def get_encoded_text(self, data_list):

        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_name)
        encoded_data = tokenizer(data_list, padding = True, truncation= True )
        
        return encoded_data


    def get_processed_data(self):
        file_name = os.listdir(self.config.raw_data_dir)[0]
        file_path = join_path(self.config.raw_data_dir,file_name)

        df = pd.read_csv(file_path, header= False)

        # encoding text

        text_list = df[self.config.text_col].to_list()
        encoded_text = self.get_encoded_text(text_list)

        # encoding the labels
        encoded_labels = self.get_encoded_labels(df[self.config.label_col])
        labels = encoded_labels.to_list()

        



